# Programming in Data Science - Final Project
## Invoices Dataset Analysis
**Team Members: Leo WINTER, Yoann SUBLET, Kellian VERVAELE KLEIN, Alvaro SERERO**

Dataset: Invoices (Kaggle)

Source: https://www.kaggle.com/datasets/cankatsrc/invoices/data

This dataset includes multiple fields such as customer details (first name, last name, email), transaction information (product ID, quantity, amount, invoice date), and additional attributes like address, city, and stock code.

### Import all needed libraries for the project:
- Pandas for data manipulation
- Plotly express for visualizations
- Dash for creating a visual and interactive dashboard interface

In [29]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html,Input, Output
from dash import callback
from prophet import Prophet
from prophet.plot import plot_plotly
import logging

## 1) Data collection and exploration

### Function to safely load CSV data from a file path

In [2]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    Function to load CSV data from a given file path safely.

    Input: 
    ------
    file_path => String, path to the CSV file

    Output: 
    ------
    dataset => pd.DataFrame containing the loaded data
    """
    try:
        df = pd.read_csv(file_path)
        print("Data loaded successfully.")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame()

### Function to process invalid first_name and last_name columns.
- In the initial dataset there are "last_name" and "first_name" columns but each one contains a combination of a first name and last name which does not make sense since a trasaction is only made by one individual person and columns should contain exactly what is described by their name.
- For example, the first line is structured as follows, which is a mistake and need to be corrected.

first_name | last_name  
Carmen Nixon | Todd Anderson

In [3]:
def name_treatment(dataset: pd.DataFrame, options: str="separate") -> pd.DataFrame:
    """
    Function to treat first_name and last_name columns in a dataset.
    
    Input:
    ---------
    - dataset => Pandas DataFrame, dataset must have first_name and last_name columns
    - options => String, options for treatment between "separate", "first" and "last"
        - "separate" (default): create two new line for each name, 
        - "first": keep only the first_name renamed as name, 
        - "last" : keep only the last_name renamed as name
    
    Output:
    ---------
    - dataset => Pandas DataFrame after treating first_name and last_name columns
    """

    if "first_name" in dataset and "last_name" in dataset:
        if options == "separate":
            value = dataset.columns.difference(['first_name','last_name']).tolist()
            new_dataset = pd.melt(dataset, id_vars=value,              
                              value_vars=['first_name', 'last_name'],
                              value_name='name')
            
            autres_colonnes = [col for col in new_dataset.columns if col not in ["name", "variable"]]
            nouvel_ordre = ["name"] + autres_colonnes
            new_dataset = new_dataset[nouvel_ordre]

        elif options == "first":
            new_dataset = dataset.drop(columns=['last_name'])
            new_dataset.rename(columns={'first_name': 'name'}, inplace=True)

        elif options == "last":
            new_dataset = dataset.drop(columns=['first_name'])
            new_dataset.rename(columns={'last_name': 'name'}, inplace=True)
        else:
            print(f"{options} is not a correct parameters of options, please write 'separate' or 'first' or 'last")
            return dataset
        return new_dataset
    else:
        return dataset

### Function to parse invoice dates:
- Convert "invoice_date" column to datetime for futural temporal manipulations.
- Extracts year, month, day, and day of week features.

In [4]:
def parse_dates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts "invoice_date" column from string to datetime.
    Extracts year, month, day, and day of week features.

    Input:
    ------
    - df (DataFrame) - dataset with invoice_date column

    Output:
    ------
    - df (DataFrame) - dataset with parsed datetime features
    """
    if 'invoice_date' not in df.columns:
        print("Column 'invoice_date' not found in DataFrame.")
        return df
    
    df = df.copy()
    df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='%d/%m/%Y', errors='coerce')
    df['year'] = df['invoice_date'].dt.year
    df['month'] = df['invoice_date'].dt.month
    df['day'] = df['invoice_date'].dt.day
    df['dayofweek'] = df['invoice_date'].dt.dayofweek
    print("Dates parsed and temporal features extracted")
    return df

### Covert all string columns of the dataset to strip whitespaces.

In [5]:
def convert_string_columns(df: pd.DataFrame) -> None:
    """
    String manipulation: Strip whitespace from object columns

    Input:
    -------
    - df => Pandas DataFrame to be processed
    Output:
    ------- 
    None (the function modifies the DataFrame in place)
    """
    string_cols = df.select_dtypes(include=['object']).columns
    for col in string_cols:
        df[col] = df[col].str.strip()

### Function to preprocess the initial loaded dataset: combines all the previous functions and returns a clean dataset.

In [6]:
def preprocess_data(df: pd.DataFrame, name_options: str ="separate") -> pd.DataFrame:
    """
    Function to preprocess the initial loaded dataset:
    - Strips whitespaces from strings using the convert_string_columns function. 
    - Converts "invoice_date" column to datetime for futural temporal manipulations.
    - Adds "revenue" column derived from "qty" and "amount" columns.
    - Create a column name using the name_treatment function correcting the first_name and last_name column

    Input:
    ---------
    - df => Pandas DataFrame to be preprocessed
    - [Optionnal] name_options (String) => options for the name_treatment function. Possible choixe "separate", "first" and "last"
    Output:
    ---------
    - df => Preprocessed Pandas DataFrame
    """
    if 'qty' in df.columns and 'amount' in df.columns:
        # Create 'revenue' column as product of 'quantity' and 'amount'
        df['revenue'] = df['qty'] * df['amount']

    df = name_treatment(df, options=name_options)
    df = parse_dates(df)
    convert_string_columns(df)

    return df

### Function for data exploration: displaying basic information on our dataset.
We can see that there is no missing or NaN data since all columns have 10000 non-null rows.

In [7]:
def explore_data(df: pd.DataFrame) -> None:
    """
    Prints key exploratory information: 
    - dataset shape (rows, columns)
    - column data types
    - missing values per column
    - description of columns
    - correlation matrix between numerical columns

    Input:
    ---------
    - df => Pandas DataFrame to be explored

    Output:
    ---------
    None (prints information to console)
    """
    print("Shape (rows, columns):", df.shape)

    print("\nColumn dtypes:")
    print(df.dtypes)

    print("\nMissing values per column:")
    print(df.isna().sum())

    print("\nBasic description of numerical columns:")
    print(df.describe())

    # Correlation matrix for numeric variables
    print("\nCorrelation matrix (numeric columns):")
    print(df[['qty', 'amount', 'revenue']].corr())

### Testing data collection, preprocessing and exploration on the Invoices dataset.

In [9]:
explore_data(preprocess_data(load_data('invoices.csv')))

Data loaded successfully.
Dates parsed and temporal features extracted
Shape (rows, columns): (20000, 15)

Column dtypes:
name                    object
address                 object
amount                 float64
city                    object
email                   object
invoice_date    datetime64[ns]
job                     object
product_id               int64
qty                      int64
revenue                float64
stock_code               int64
year                     int32
month                    int32
day                      int32
dayofweek                int32
dtype: object

Missing values per column:
name            0
address         0
amount          0
city            0
email           0
invoice_date    0
job             0
product_id      0
qty             0
revenue         0
stock_code      0
year            0
month           0
day             0
dayofweek       0
dtype: int64

Basic description of numerical columns:
             amount                invoice_date

## 2) Querying the dataset

### Indicator 1: Total revenue by city
Identifies the most profitable geographic locations by aggregating total revenue by city.

In [10]:
def calculate_revenue_by_city(df: pd.DataFrame, n: int=10) -> pd.DataFrame:
    """
    Groups invoices by city and calculates total revenue.
    Returns top n cities ranked by total transaction amount.
    
    Inputs:
    ---------
    - df (DataFrame): Input dataset
    - [Optionnal] n (int): Number of top cities to return

    Output: 
    ---------
    grouped_df (DataFrame) - Contains 'Country' and 'TotalAmount'.
    """
    revenue_by_city = df.groupby('city').agg({
        'amount': 'sum',
        'product_id': 'count'
    }).reset_index()

    revenue_by_city.columns = ['City', 'Total_Revenue', 'Transaction_Count']
    revenue_by_city = revenue_by_city.sort_values('Total_Revenue', ascending=False)
    
    print(revenue_by_city.head(n))
    return revenue_by_city

To do => Explication of the indicator here

In [ ]:
def indicator_top_cities(df: pd.DataFrame, n: int = 10) -> pd.DataFrame:

    city_rev = (
        df.groupby('city', as_index=False)['revenue']
        .sum()
        .rename(columns={'revenue': 'total_revenue'})
        .sort_values('total_revenue', ascending=False)
        .head(n)
    )
    return city_rev

In [19]:
def monthly_revenue(df: pd.DataFrame) -> pd.DataFrame:
    """Get monthly revenue."""
    monthly_revenue_df = df.set_index('invoice_date').resample('M')['revenue'].sum().to_frame('monthly_revenue')
    return monthly_revenue_df

monthly_revenue(df)

/var/folders/cr/3y3t2vr97xsf_f_y_xhw2jv00000gn/T/ipykernel_90892/4161595773.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



,monthly_revenue
invoice_date,
1970-01-31,3878.93
1970-02-28,6050.15
1970-03-31,5527.01
1970-04-30,2050.53
1970-05-31,2779.05
...,...
2021-09-30,5605.31
2021-10-31,3333.89
2021-11-30,5293.00


Function to make temporal prediction on the dataset

In [34]:
def temporal_prediction(df: pd.DataFrame, time: str="year",periods: int = 10):
    """
    Use Prophet model to make a temporal prediction of the revenue.

    Inputs:
    ---------
    - df (DataFrame): Input dataset
    - [Optionnal] time (str): options for the prediction between "year" and "month".  
        - "year" (default): use the year column of the dataset to make the prediction.  
        - "month": use the month column of the dataset to make the prediction.  
    - [Optionnal] periods (int): The period to calculate the future date. Default 10.

    Outputs: 
    --------
    - new_df (DataFrame) - Contains 'time', 'original_revenue' and 'predicted_revenue'.
    - model (Prophet) - Prophet model trained on the dataset and used for the prediction
    - prediction (DataFrame) - Future prediction made by the model
    """
    dataset = df.copy()
    if time == "year":
        # A mettre ailleur la transformation en datetime ?
        dataset['year'] = pd.to_datetime(dataset['year'], format='%Y')

        freq = 'YE'
        dataset = dataset.groupby('year')['revenue'].sum().reset_index()
        dataset.rename(columns={'year': 'ds','revenue': 'y'}, inplace=True)

    elif time == "month":
        # A mettre ailleur la transformation en datetime ?
        dataset['month'] = dataset['year'].astype(str) + '-' + dataset['month'].astype(str).str.zfill(2)
        dataset['month'] = pd.to_datetime(dataset['month'], format='%Y-%m')

        freq = 'ME'
        dataset = dataset.groupby('month')['revenue'].sum().reset_index()
        dataset.rename(columns={'month': 'ds','revenue': 'y'}, inplace=True)
#   elif time == "day":

    else:
        print("Erreur: L'option 'time' doit être 'year' ou 'month'.")
        return df,None,None

    # Put cmdstanpy log ouput at ERROR to not have the output when the function is used
    logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

    # Create a Prophet model to make prediction
    model = Prophet()
    model.fit(dataset)

    future_dates = model.make_future_dataframe(periods=periods, freq=freq)
    prediction = model.predict(future_dates)

    new_df = pd.merge(dataset[['ds', 'y']], prediction[['ds', 'yhat']], on='ds', how='outer')
    new_df.rename(columns={'y': 'original_revenue','yhat': 'predicted_revenue', 'ds': 'time'}, inplace=True)
    
    return new_df, model,prediction

    

Function to create a visualization based on a temporal prediction

In [12]:
def display_temporal_prediction(df: pd.DataFrame, model,prediction,options: str = "prophet"):
    """
    Create a visualization of the a dataset with temporal prediction 
    either with the dataset or with the prediction model.

    Inputs:
    ---------  
    - df (DataFrame): Input dataset.
    - model (Prophet): Prophet model trained on the dataset and used for the prediction.
    - prediction (DataFrame): Future prediction made by the model.
    - [Optionnal] options (str): options for the visualization between "ploty" and "prophet". 
        - "prophet": leo.   
        - "ploty" (default): leo.  

    Outputs: 
    --------
    - fig (Figure) - A figure containing the temporal visualization.
    """
    if options=="ploty":
        if 'predicted_revenue' in df.columns and 'original_revenue' in df.columns:
            fig = px.area()
            fig.add_scatter(x=df.index, y=df["original_revenue"], mode='lines', line=dict(color='blue'), name="original")
            fig.add_scatter(x=df.index ,y=df["predicted_revenue"], mode='lines', line=dict(color='green'), name="prediction")
            fig.update_layout(title="Prediction", xaxis_title="Date", yaxis_title="Revenue")
        else:
            print("Error, the prediction  was not found in the dataset")
            fig = None

    elif options == "prophet":
        if model is not None:
            fig = plot_plotly(model, prediction)
        else:
            print("Error, the prediction model was not found")
            fig = None

    # elif options == "3":
    
    return fig

## 3) Dash visualization

In [32]:
def create_dashboard(df: pd.DataFrame) -> Dash:
    # Indicator 1
    revenue_by_city = calculate_revenue_by_city(df, n=10)
    
    figure_city_revenue = px.bar(
        revenue_by_city.head(15),
        x='City',
        y='Total_Revenue',
        title='Top 15 Cities by Revenue',
        labels={'Total_Revenue': 'Total Revenue ($)', 'City': 'City Name'},
        color='Transaction_Count',
        color_continuous_scale='Blues'
    ).update_layout(
        xaxis_tickangle=-45,
        height=400
    )

    # Indicator 2
    figure_pred_year = []
    figure_pred_month = []
    time_pred = [1900,2015,2010,2000,1990,1980]
    for i in time_pred:
        dataset = df.copy()
        dataset = dataset[dataset["year"]>i]
        data_y, model_y,predictions_y = temporal_prediction(dataset,time="year")
        figure_pred_year.append(display_temporal_prediction(data_y,model_y,predictions_y))
        data_m,model_m,predictions_m = temporal_prediction(dataset,time="month")
        figure_pred_month.append(display_temporal_prediction(data_m,model_m,predictions_m))
    year_to_index = {
    "YA": 0, "MA": 0,    # 1900 (All)
    "Y2015": 1, "M2015": 1,
    "Y2010": 2, "M2010": 2,
    "Y2000": 3, "M2000": 3,
    "Y1990": 4, "M1990": 4,
    "Y1980": 5, "M1980": 5}
    
    # Initialize the Dash app
    app = Dash(__name__)

    app.layout = html.Div([
        # Header
        html.Div([
            html.H1("Invoices Dashboard – Group members: Alvaro SERERO, Leo WINTER, Yoann SUBLET, Kellian VERVAELE KLEIN"),
        ]),

        # Dashboard content
        # Indicator 1: Top Cities by Revenue
        html.Div([
            html.H3('Indicator 1: Top Cities by Total Revenue', 
                    style={'color': '#2980b9', 'marginBottom': 15}),
            html.P('Grouping Query - Aggregates total revenue and transaction count per city',
                    style={'fontSize': 13, 'color': '#7f8c8d', 'marginBottom': 15}),
            dcc.Graph(figure=figure_city_revenue),
        ]),

        html.Div([
                html.H3('Indicator 2: Prediction of future revenue', 
                    style={'color': '#2980b9', 'marginBottom': 15}),
                html.P('Predict the future by taking information from past time',
                    style={'fontSize': 13, 'color': '#7f8c8d', 'marginBottom': 15}),
                dcc.Graph(id='graph'),
                dcc.Dropdown(options=[{"label": "All Year", "value": "YA"},{"label": "2015 Year", "value": "Y2015"},
                                      {"label": "2010 Year", "value": "Y2010"}, {"label": "2000 Year", "value": "Y2000"},
                                      {"label": "1990 Year", "value": "Y1990"},{"label": "1980 Year", "value": "Y1980"},
                                      {"label": "All Month", "value": "MA"},{"label": "2015 Month", "value": "M2015"},
                                      {"label": "2010 Month", "value": "M2010"}, {"label": "2000 Month", "value": "M2000"},
                                      {"label": "1990 Month", "value": "M1990"},{"label": "1980 Month", "value": "M1980"}],
                                        value="YA", id='dropdown')
                ])
    ])

    @callback(
    Output('graph', 'figure'),
    Input('dropdown', 'value'))
    def update_temporal_graph(selected_value):
        index = year_to_index[selected_value]
        if selected_value.startswith("Y"):
            fig_dash = figure_pred_year[index]
        elif selected_value.startswith("M"):
            fig_dash = figure_pred_month[index]
        return fig_dash
    return app

In [35]:
def main():
    file_path = "invoices.csv"
    df = load_data(file_path)
    df = preprocess_data(df)
    explore_data(df)

    app = create_dashboard(df)
    app.run()

if __name__ == "__main__":
    main()

Data loaded successfully.
Dates parsed and temporal features extracted
Shape (rows, columns): (20000, 15)

Column dtypes:
name                    object
address                 object
amount                 float64
city                    object
email                   object
invoice_date    datetime64[ns]
job                     object
product_id               int64
qty                      int64
revenue                float64
stock_code               int64
year                     int32
month                    int32
day                      int32
dayofweek                int32
dtype: object

Missing values per column:
name            0
address         0
amount          0
city            0
email           0
invoice_date    0
job             0
product_id      0
qty             0
revenue         0
stock_code      0
year            0
month           0
day             0
dayofweek       0
dtype: int64

Basic description of numerical columns:
             amount                invoice_date

In [18]:
file_path = "invoices.csv"
dataset = load_data(file_path)
dataset = preprocess_data(dataset)
dataset = dataset[dataset["year"]>2010]
dataset, model, pred = temporal_prediction(dataset,time="month")
dataset.head()



18:35:01 - cmdstanpy - INFO - Chain [1] start processing


Data loaded successfully.
Dates parsed and temporal features extracted


18:35:01 - cmdstanpy - INFO - Chain [1] done processing


,time,original_revenue,predicted_revenue
0,2011-01-01,8330.52,7600.434011
1,2011-02-01,8489.40,8418.968870
2,2011-03-01,8415.04,8407.216700
3,2011-04-01,12473.16,7564.877258
4,2011-05-01,9097.32,7773.558581


In [21]:
display_temporal_prediction(dataset, model, pred)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': 'black', 'size': 4},
              'mode': 'markers',
              'name': 'Actual',
              'type': 'scatter',
              'x': array(['2011-01-01T00:00:00.000000000', '2011-02-01T00:00:00.000000000',
                          '2011-03-01T00:00:00.000000000', '2011-04-01T00:00:00.000000000',
                          '2011-05-01T00:00:00.000000000', '2011-06-01T00:00:00.000000000',
                          '2011-07-01T00:00:00.000000000', '2011-08-01T00:00:00.000000000',
                          '2011-09-01T00:00:00.000000000', '2011-10-01T00:00:00.000000000',
                          '2011-11-01T00:00:00.000000000', '2011-12-01T00:00:00.000000000',
                          '2012-01-01T00:00:00.000000000', '2012-02-01T00:00:00.000000000',
                          '2012-03-01T00:00:00.000000000', '2012-04-01T00:00:00.000000000',
                          '2012-05-01T00:00:00.000000000', '2012-06-01T00:00:00.000000000',
                          '2012-07-01T00:00:00.000000000', '2012-08-01T00:00:00.000000000',
                          '2012-09-01T00:00:00.000000000', '2012-10-01T00:00:00.000000000',
                          '2012-11-01T00:00:00.000000000', '2012-12-01T00:00:00.000000000',
                          '2013-01-01T00:00:00.000000000', '2013-02-01T00:00:00.000000000',
                          '2013-03-01T00:00:00.000000000', '2013-04-01T00:00:00.000000000',
                          '2013-05-01T00:00:00.000000000', '2013-06-01T00:00:00.000000000',
                          '2013-07-01T00:00:00.000000000', '2013-08-01T00:00:00.000000000',
                          '2013-09-01T00:00:00.000000000', '2013-10-01T00:00:00.000000000',
                          '2013-11-01T00:00:00.000000000', '2013-12-01T00:00:00.000000000',
                          '2014-01-01T00:00:00.000000000', '2014-02-01T00:00:00.000000000',
                          '2014-03-01T00:00:00.000000000', '2014-04-01T00:00:00.000000000',
                          '2014-05-01T00:00:00.000000000', '2014-06-01T00:00:00.000000000',
                          '2014-07-01T00:00:00.000000000', '2014-08-01T00:00:00.000000000',
                          '2014-09-01T00:00:00.000000000', '2014-10-01T00:00:00.000000000',
                          '2014-11-01T00:00:00.000000000', '2014-12-01T00:00:00.000000000',
                          '2015-01-01T00:00:00.000000000', '2015-02-01T00:00:00.000000000',
                          '2015-03-01T00:00:00.000000000', '2015-04-01T00:00:00.000000000',
                          '2015-05-01T00:00:00.000000000', '2015-06-01T00:00:00.000000000',
                          '2015-07-01T00:00:00.000000000', '2015-08-01T00:00:00.000000000',
                          '2015-09-01T00:00:00.000000000', '2015-10-01T00:00:00.000000000',
                          '2015-11-01T00:00:00.000000000', '2015-12-01T00:00:00.000000000',
                          '2016-01-01T00:00:00.000000000', '2016-02-01T00:00:00.000000000',
                          '2016-03-01T00:00:00.000000000', '2016-04-01T00:00:00.000000000',
                          '2016-05-01T00:00:00.000000000', '2016-06-01T00:00:00.000000000',
                          '2016-07-01T00:00:00.000000000', '2016-08-01T00:00:00.000000000',
                          '2016-09-01T00:00:00.000000000', '2016-10-01T00:00:00.000000000',
                          '2016-11-01T00:00:00.000000000', '2016-12-01T00:00:00.000000000',
                          '2017-01-01T00:00:00.000000000', '2017-02-01T00:00:00.000000000',
                          '2017-03-01T00:00:00.000000000', '2017-04-01T00:00:00.000000000',
                          '2017-05-01T00:00:00.000000000', '2017-06-01T00:00:00.000000000',
                          '2017-07-01T00:00:00.000000000', '2017-08-01T00:00:00.000000000',
                          '2017-09-01T00:00:00.000000000', '2017-10-01T00:00:00.000000000',
                          '2017-11-01T00:00:00.000000000', '2017-1